# Como definir un problema en pytorch sin usar classes

In [17]:
import torch
import torch.optim as optim

# Función para definir el modelo
def simple_model(x, weights, bias):
    return torch.matmul(x, weights) + bias

# Función de pérdida personalizada
def custom_loss(y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)

# Función para calcular los gradientes manualmente de forma analítica
def calculate_gradients(x, y, weights, bias):
    with torch.no_grad():
        gradient = torch.mean(2 * (simple_model(x, weights, bias) - y) * x, dim=0)
        bias_gradient = torch.mean(2 * (simple_model(x, weights, bias) - y))

    return gradient, bias_gradient

# Inicializar los parámetros del modelo
weights = torch.randn(1, 1, requires_grad=True)
bias = torch.randn(1, requires_grad=True)

# Datos de ejemplo
x_train = torch.tensor([[1.0], [2.0], [3.0], [4.0], [5.0]])
y_train = torch.tensor([[2.0], [4.0], [6.0], [8.0], [10.0]])

# Definir el optimizador
#optimizer = optim.SGD([weights, bias], lr=0.1)
optimizer = optim.SGD([weights, bias], lr=0.01, momentum=0.9)

# Ciclo de entrenamiento
epochs = 200
batch_size = 3
num_batches = len(x_train) // batch_size

for epoch in range(epochs):
    # Iterar sobre los lotes de datos
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        # Obtener el lote actual
        x_batch = x_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]

        # Paso de adelante: Calcular la predicción y la pérdida
        y_pred = simple_model(x_batch, weights, bias)
        loss = custom_loss(y_pred, y_batch)

        # Calcular los gradientes manualmente de forma analítica
        weight_gradient, bias_gradient = calculate_gradients(x_batch, y_batch, weights, bias)

        # Actualizar los parámetros del modelo utilizando el optimizador
        optimizer.zero_grad()
        weights.grad = weight_gradient.reshape_as(weights)
        bias.grad = bias_gradient.reshape_as(bias)
        optimizer.step()

    # Imprimir la pérdida
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Evaluar el modelo entrenado
with torch.no_grad():
    print("Predicciones después del entrenamiento:")
    y_pred = simple_model(x_train, weights, bias)
    for i, (predicted, true) in enumerate(zip(y_pred, y_train)):
        print(f'Entrada: {x_train[i].item()}, Predicción: {predicted.item()}, Valor real: {true.item()}')


Epoch [10/200], Loss: 0.7478073239326477
Epoch [20/200], Loss: 0.2538773715496063
Epoch [30/200], Loss: 0.07556147128343582
Epoch [40/200], Loss: 0.01932181976735592
Epoch [50/200], Loss: 0.00405034190043807
Epoch [60/200], Loss: 0.0006382136489264667
Epoch [70/200], Loss: 8.205812628148124e-05
Epoch [80/200], Loss: 3.938790177926421e-05
Epoch [90/200], Loss: 3.848583946819417e-05
Epoch [100/200], Loss: 2.7724838219000958e-05
Epoch [110/200], Loss: 1.5634976080036722e-05
Epoch [120/200], Loss: 7.470439868484391e-06
Epoch [130/200], Loss: 3.164819872836233e-06
Epoch [140/200], Loss: 1.2282999932722305e-06
Epoch [150/200], Loss: 4.4860919956590806e-07
Epoch [160/200], Loss: 1.598371994759873e-07
Epoch [170/200], Loss: 5.812311698605299e-08
Epoch [180/200], Loss: 2.2797792098572245e-08
Epoch [190/200], Loss: 9.866293559923633e-09
Epoch [200/200], Loss: 4.5815986560171496e-09
Predicciones después del entrenamiento:
Entrada: 1.0, Predicción: 2.000088930130005, Valor real: 2.0
Entrada: 2.0, 

# Como usar LBFGS full batch

In [19]:
import torch
import torch.optim as optim

# Función para definir el modelo
def simple_model(x, weights, bias):
    return torch.matmul(x, weights) + bias

# Función de pérdida personalizada
def custom_loss(y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)

# Función para calcular los gradientes manualmente de forma analítica
def calculate_gradients(x, y, weights, bias):
    with torch.no_grad():
        gradient = torch.mean(2 * (simple_model(x, weights, bias) - y) * x, dim=0)
        bias_gradient = torch.mean(2 * (simple_model(x, weights, bias) - y))

    return gradient, bias_gradient

# Inicializar los parámetros del modelo
weights = torch.randn(1, 1, requires_grad=True)
bias = torch.randn(1, requires_grad=True)

# Datos de ejemplo
x_train = torch.tensor([[1.0], [2.0], [3.0], [4.0], [5.0]])
y_train = torch.tensor([[2.0], [4.0], [6.0], [8.0], [10.0]])

# Definir el optimizador LBFGS
optimizer = optim.LBFGS([weights, bias], lr=0.1)

# Función para realizar un paso de optimización con LBFGS
def closure():
    optimizer.zero_grad()
    y_pred = simple_model(x_train, weights, bias)
    loss = custom_loss(y_pred, y_train)

    # Calcular los gradientes manualmente
    weight_gradient, bias_gradient = calculate_gradients(x_train, y_train, weights, bias)

    # Asignar los gradientes calculados manualmente
    weights.grad = weight_gradient.reshape_as(weights)
    bias.grad = bias_gradient.reshape_as(bias)

    return loss

# Ciclo de entrenamiento
epochs = 100

for epoch in range(epochs):
    optimizer.step(closure)

    # Imprimir la pérdida
    if (epoch+1) % 10 == 0:
        with torch.no_grad():
            loss = closure()
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Evaluar el modelo entrenado
with torch.no_grad():
    print("Predicciones después del entrenamiento:")
    y_pred = simple_model(x_train, weights, bias)
    for i, (predicted, true) in enumerate(zip(y_pred, y_train)):
        print(f'Entrada: {x_train[i].item()}, Predicción: {predicted.item()}, Valor real: {true.item()}')


Epoch [10/100], Loss: 1.8604737750393952e-09
Epoch [20/100], Loss: 4.32783059389763e-10
Epoch [30/100], Loss: 4.32783059389763e-10
Epoch [40/100], Loss: 4.32783059389763e-10
Epoch [50/100], Loss: 4.32783059389763e-10
Epoch [60/100], Loss: 4.32783059389763e-10
Epoch [70/100], Loss: 4.32783059389763e-10
Epoch [80/100], Loss: 4.32783059389763e-10
Epoch [90/100], Loss: 4.32783059389763e-10
Epoch [100/100], Loss: 4.32783059389763e-10
Predicciones después del entrenamiento:
Entrada: 1.0, Predicción: 2.000035285949707, Valor real: 2.0
Entrada: 2.0, Predicción: 4.000025272369385, Valor real: 4.0
Entrada: 3.0, Predicción: 6.0000152587890625, Valor real: 6.0
Entrada: 4.0, Predicción: 8.000005722045898, Valor real: 8.0
Entrada: 5.0, Predicción: 9.999996185302734, Valor real: 10.0


# Como usar LBFGS en batches

In [16]:
import torch
import torch.optim as optim

# Función para definir el modelo
def simple_model(x, weights, bias):
    return torch.matmul(x, weights) + bias

# Función de pérdida personalizada
def custom_loss(y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)

# Función para calcular los gradientes manualmente de forma analítica
def calculate_gradients(x, y, weights, bias):
    with torch.no_grad():
        gradient = torch.mean(2 * (simple_model(x, weights, bias) - y) * x, dim=0)
        bias_gradient = torch.mean(2 * (simple_model(x, weights, bias) - y))

    return gradient, bias_gradient

# Inicializar los parámetros del modelo
weights = torch.randn(1, 1, requires_grad=True)
bias = torch.randn(1, requires_grad=True)

# Datos de ejemplo
x_train = torch.tensor([[1.0], [2.0], [3.0], [4.0], [5.0]])
y_train = torch.tensor([[2.0], [4.0], [6.0], [8.0], [10.0]])

# Definir el optimizador LBFGS
optimizer = optim.LBFGS([weights, bias], lr=0.01)

# Función para realizar un paso de optimización con LBFGS
def closure():
    optimizer.zero_grad()
    total_loss = 0
    for i in range(0, len(x_train), batch_size):
        x_batch = x_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        y_pred = simple_model(x_batch, weights, bias)
        loss = custom_loss(y_pred, y_batch)
        total_loss += loss.item()

        # Calcular los gradientes manualmente
        weight_gradient, bias_gradient = calculate_gradients(x_batch, y_batch, weights, bias)

        # Asignar los gradientes calculados manualmente
        weights.grad = weight_gradient.reshape_as(weights)
        bias.grad = bias_gradient.reshape_as(bias)

    # Retornar el total de la pérdida para el cálculo de la media
    return total_loss / len(x_train)

# Ciclo de entrenamiento
epochs = 100
batch_size = 3

for epoch in range(epochs):
    optimizer.step(closure)

    # Imprimir la pérdida
    if (epoch+1) % 10 == 0:
        with torch.no_grad():
            loss = closure()
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss}')

# Evaluar el modelo entrenado
with torch.no_grad():
    print("Predicciones después del entrenamiento:")
    y_pred = simple_model(x_train, weights, bias)
    for i, (predicted, true) in enumerate(zip(y_pred, y_train)):
        print(f'Entrada: {x_train[i].item()}, Predicción: {predicted.item()}, Valor real: {true.item()}')


Epoch [10/100], Loss: 1.658161497116089
Epoch [20/100], Loss: 0.15399158596992493
Epoch [30/100], Loss: 0.06731706149876118
Epoch [40/100], Loss: 0.0015108366613276302
Epoch [50/100], Loss: 2.5963342795876088e-05
Epoch [60/100], Loss: 4.7185599214571996e-07
Epoch [70/100], Loss: 4.678699667692854e-08
Epoch [80/100], Loss: 3.849015399737254e-08
Epoch [90/100], Loss: 3.1645950571146384e-08
Epoch [100/100], Loss: 2.6175644052273128e-08
Predicciones después del entrenamiento:
Entrada: 1.0, Predicción: 1.9995261430740356, Valor real: 2.0
Entrada: 2.0, Predicción: 3.9996628761291504, Valor real: 4.0
Entrada: 3.0, Predicción: 5.999799728393555, Valor real: 6.0
Entrada: 4.0, Predicción: 7.999936580657959, Valor real: 8.0
Entrada: 5.0, Predicción: 10.00007438659668, Valor real: 10.0
